## Round-Robin Group Chat
`RoundRobinGroupChat` is a simple team that allows all agents to share context and take turns to respond in a round-robin fashion. 

On its turn, each agent broadcasts its response to all other agents in the team, so **all agents have the same context**.

We will start by creating a team with a single AssistantAgent agent and TextMentionTermination termination condition that stops the team when a word is detected.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

print(os.getenv('CHAT_MODEL'))

gpt-4o-mini


In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

base_url = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01").base_url

api_endpoint = f'https://{base_url.host}/'

api_key = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01").api_key

deployment_name = os.environ["CHAT_MODEL"]

aoai_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=api_endpoint,
    model="gpt-4o-mini",
    azure_deployment=deployment_name,
    api_key=api_key,
    api_version="2024-06-01"
)

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination


# Define a tool that gets the weather for a city.
async def get_weather(city: str) -> str:
    """Get the weather for a city."""
    return f"The weather in {city} is 72 degrees and Sunny."


# Create an assistant agent.
weather_agent = AssistantAgent(
    "assistant",
    model_client=aoai_client,
    tools=[get_weather],
    system_message="Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition.
text_termination = TextMentionTermination("TERMINATE")

# Create a single-agent team.
single_agent_team = RoundRobinGroupChat(
    [weather_agent], termination_condition=text_termination)

In [4]:
async def run_team() -> None:
    result = await single_agent_team.run(task="What is the weather in New York?")
    print(result)


# Use `asyncio.run(run_team())` when running in a script.
await run_team()

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=16), metadata={}, content=[FunctionCall(id='call_whSX5zJgCP1QMIjdsSaS4XB7', arguments='{"city":"New York"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', name='get_weather', call_id='call_whSX5zJgCP1QMIjdsSaS4XB7', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='assistant', models_usage=None, metadata={}, content='The weather in New York is 72 degrees and Sunny.', type='ToolCallSummaryMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=107, completion_tokens=16), metadata={}, content='The weather in New York

In [5]:
await single_agent_team.reset()  # Reset the team for the next run.

## Streaming Team Messages
Similar to agent’s `on_messages_stream()` method, you can stream the team’s messages by calling the `run_stream()` method. 

It will return a generator that yields the messages produced by the agents in the team as they are generated, and the last item will be the task result.

In [6]:
from autogen_agentchat.base import TaskResult


async def run_team_stream() -> None:
    async for message in single_agent_team.run_stream(task="What is the weather in New York?"):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)


# Use `asyncio.run(run_team_stream())` when running in a script.
await run_team_stream()

source='user' models_usage=None metadata={} content='What is the weather in New York?' type='TextMessage'
source='assistant' models_usage=RequestUsage(prompt_tokens=70, completion_tokens=16) metadata={} content=[FunctionCall(id='call_0VoTIfV2I2yxOOk9HPWIxi83', arguments='{"city":"New York"}', name='get_weather')] type='ToolCallRequestEvent'
source='assistant' models_usage=None metadata={} content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', name='get_weather', call_id='call_0VoTIfV2I2yxOOk9HPWIxi83', is_error=False)] type='ToolCallExecutionEvent'
source='assistant' models_usage=None metadata={} content='The weather in New York is 72 degrees and Sunny.' type='ToolCallSummaryMessage'
source='assistant' models_usage=RequestUsage(prompt_tokens=107, completion_tokens=16) metadata={} content='The weather in New York is 72 degrees and sunny. TERMINATE' type='TextMessage'
Stop Reason: Text 'TERMINATE' mentioned


In [7]:
from autogen_agentchat.ui import Console

# Use `asyncio.run(single_agent_team.reset())` when running in a script.
await single_agent_team.reset()  # Reset the team for the next run.
# Use `asyncio.run(single_agent_team.run_stream(task="What is the weather in Seattle?"))` when running in a script.
await Console(
    single_agent_team.run_stream(task="What is the weather in Seattle?")
)  # Stream the messages to the console.

---------- user ----------
What is the weather in Seattle?
---------- assistant ----------
[FunctionCall(id='call_v5Y61QNh657OiIJIw53ZbjFc', arguments='{"city":"Seattle"}', name='get_weather')]
---------- assistant ----------
[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', name='get_weather', call_id='call_v5Y61QNh657OiIJIw53ZbjFc', is_error=False)]
---------- assistant ----------
The weather in Seattle is 72 degrees and Sunny.
---------- assistant ----------
The weather in Seattle is 72 degrees and sunny.
---------- assistant ----------

---------- assistant ----------

---------- assistant ----------
[FunctionCall(id='call_VjkxwoXtw6QK2kf8PEhYNaT6', arguments='{"city":"Seattle"}', name='get_weather')]
---------- assistant ----------
[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', name='get_weather', call_id='call_VjkxwoXtw6QK2kf8PEhYNaT6', is_error=False)]
---------- assistant ----------
The weather in Seattle is

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the weather in Seattle?', type='TextMessage'), ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=69, completion_tokens=15), metadata={}, content=[FunctionCall(id='call_v5Y61QNh657OiIJIw53ZbjFc', arguments='{"city":"Seattle"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', name='get_weather', call_id='call_v5Y61QNh657OiIJIw53ZbjFc', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='assistant', models_usage=None, metadata={}, content='The weather in Seattle is 72 degrees and Sunny.', type='ToolCallSummaryMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=104, completion_tokens=13), metadata={}, content='The weather in Seattle is 7

## Reflection Pattern
Now we will create a team with two agents that implements the Reflection pattern, which is a multi-agent design pattern that uses a critic agent to evaluate the responses of a primary agent.

See how the reflection pattern works using the Core API.

In this example, we will use the `AssistantAgent` agent class for both the primary and critic agents. We will use both the `TextMentionTermination` and `MaxMessageTermination` termination conditions together to stop the team.

In [8]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination


# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=aoai_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=aoai_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(5)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Create a team with the primary and critic agents.
reflection_team = RoundRobinGroupChat(
    [primary_agent, critic_agent], termination_condition=termination)

In [9]:
# Use `asyncio.run(Console(reflection_team.run_stream(task="Write a short poem about fall season.")))` when running in a script.
await Console(
    reflection_team.run_stream(task="Write a short poem about fall season.")
)

---------- user ----------
Write a short poem about fall season.
---------- primary ----------
In the crisp air, leaves gently sway,  
Gold and amber mark the day.  
Pumpkins gleam in twilight’s glow,  
Harvest time, a bountiful show.  

Whispers of wind through trees that sigh,  
As they shed their garments, bright and spry.  
Sweaters pulled tight, a warm embrace,  
Nature's artwork, a vibrant lace.  

Hot cider brews, and stories unfold,  
By fireside warmth, the nights grow bold.  
With each fading light, a soft goodbye,  
To summer’s warmth, as the seasons fly.
---------- critic ----------
Your poem beautifully captures the essence of the fall season with vivid imagery and a warm, inviting tone. The use of sensory details, like "crisp air" and "hot cider brews," effectively immerses the reader in the experience of autumn. 

A couple of suggestions to enhance the poem further:
1. Consider using more varied line lengths or structures to create a dynamic rhythm.
2. You might explore 

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=27, completion_tokens=121), metadata={}, content="In the crisp air, leaves gently sway,  \nGold and amber mark the day.  \nPumpkins gleam in twilight’s glow,  \nHarvest time, a bountiful show.  \n\nWhispers of wind through trees that sigh,  \nAs they shed their garments, bright and spry.  \nSweaters pulled tight, a warm embrace,  \nNature's artwork, a vibrant lace.  \n\nHot cider brews, and stories unfold,  \nBy fireside warmth, the nights grow bold.  \nWith each fading light, a soft goodbye,  \nTo summer’s warmth, as the seasons fly.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=165, completion_tokens=117), metadata={}, content='Your poem beautifully captures the essence of the fall season with vivid imagery and a warm, inv

### Resuming Team
Let’s run the team again with a new task while keeping the context about the previous task.

In [10]:
# Write the poem in Chinese Tang poetry style.
# Use `asyncio.run(Console(reflection_team.run_stream(task="Rewrite the poem in Korean")))` when running in a script.
await Console(reflection_team.run_stream(task="Rewrite the poem in Korean"))

---------- user ----------
Rewrite the poem in Korean
---------- primary ----------
Here's a translation of the poem into Korean:

---

상쾌한 공기 속에 나뭇잎이 춤추고,  
황금과 호박색의 가장자리가 하루를 장식해.  
호박은 말없이 떠돌고,  
수확의 속삭임이 비밀을 밝혀내.  

바람의 속삭임이 나무 사이를 꿰뚫고,  
부드럽게 망토를 벗어내네.  
스웨터는 따스한 포옹을 이루고,  
자연의 캔버스, 화려한 레이스.  

해처럼 날이 점점 저물고,  
모닥불 옆에서 반짝이는 이야기가 생겨나.  
미소 짓는 황혼의 부드러운 빛과 함께,  
여름의 따스함에 애틋한 작별을 하네.  

---

I hope you enjoy this Korean version of the poem!
---------- critic ----------
The Korean translation of your poem beautifully captures the essence of the original while maintaining the poetic flow. The imagery and emotional resonance remain strong, making it evocative for Korean readers as well.

Here are a couple of small suggestions for improvement:
1. In the line "호박은 말없이 떠돌고," consider replacing "말없이" with something like "떠다니며" (floating), which may enhance the imagery of pumpkins drifting.
2. The phrase "화려한 레이스" could also be expressed as "아름다운 레이스" (beautiful lace) for a slightly softer co

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Rewrite the poem in Korean', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=550, completion_tokens=213), metadata={}, content="Here's a translation of the poem into Korean:\n\n---\n\n상쾌한 공기 속에 나뭇잎이 춤추고,  \n황금과 호박색의 가장자리가 하루를 장식해.  \n호박은 말없이 떠돌고,  \n수확의 속삭임이 비밀을 밝혀내.  \n\n바람의 속삭임이 나무 사이를 꿰뚫고,  \n부드럽게 망토를 벗어내네.  \n스웨터는 따스한 포옹을 이루고,  \n자연의 캔버스, 화려한 레이스.  \n\n해처럼 날이 점점 저물고,  \n모닥불 옆에서 반짝이는 이야기가 생겨나.  \n미소 짓는 황혼의 부드러운 빛과 함께,  \n여름의 따스함에 애틋한 작별을 하네.  \n\n---\n\nI hope you enjoy this Korean version of the poem!", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=780, completion_tokens=148), metadata={}, content='The Korean translation of your poem beautifully captures the essence of the original while maintaining the poetic flow. The imagery and emotional resonance remain strong, making it evocative for Korean readers as w

In [11]:
# Write the poem in Spanish.
# Use `asyncio.run(Console(reflection_team.run_stream(task="Write the poem in Chinese.")))` when running in a script.
await Console(reflection_team.run_stream(task="Write the poem in Chinese."))

---------- user ----------
Write the poem in Chinese.
---------- primary ----------
Here’s the poem translated into Chinese:

---

在清新的空气中，树叶轻舞，  
金色与琥珀色装点着这一天。  
南瓜漂浮，传递着未说的思绪，  
丰收的低语揭开秘密的面纱。  

风的低语穿梭于树间，  
轻柔地褪去它们的外衣。  
毛衣紧裹，温暖相拥，  
大自然的画布，如美丽的蕾丝。  

像太阳，日子渐渐消逝，  
在篝火旁，闪烁的故事浮现。  
伴随着微笑的黄昏柔和的光，  
我们依依不舍，告别夏日的温暖。  

---

I hope you enjoy this Chinese version of the poem!
---------- critic ----------
Your Chinese translation of the poem is beautifully done! The imagery is vivid, and you’ve successfully captured the essence and emotion of the original piece. The flow of the lines works well in Chinese, making it engaging for readers.

A couple of minor suggestions for enhancement:
1. In the line "南瓜漂浮，传递着未说的思绪," you might consider changing "传递" to "诉说," which could convey a more intimate sharing of unspoken thoughts.
2. The phrase "大自然的画布，如美丽的蕾丝" could be modified to "自然的画布，如同美丽的蕾丝" for a slightly softer and more poetic rhythm.

Overall, it's an excellent translation! Let me know if yo

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write the poem in Chinese.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=1254, completion_tokens=185), metadata={}, content='Here’s the poem translated into Chinese:\n\n---\n\n在清新的空气中，树叶轻舞，  \n金色与琥珀色装点着这一天。  \n南瓜漂浮，传递着未说的思绪，  \n丰收的低语揭开秘密的面纱。  \n\n风的低语穿梭于树间，  \n轻柔地褪去它们的外衣。  \n毛衣紧裹，温暖相拥，  \n大自然的画布，如美丽的蕾丝。  \n\n像太阳，日子渐渐消逝，  \n在篝火旁，闪烁的故事浮现。  \n伴随着微笑的黄昏柔和的光，  \n我们依依不舍，告别夏日的温暖。  \n\n---\n\nI hope you enjoy this Chinese version of the poem!', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=1456, completion_tokens=168), metadata={}, content='Your Chinese translation of the poem is beautifully done! The imagery is vivid, and you’ve successfully captured the essence and emotion of the original piece. The flow of the lines works well in Chinese, making it engaging for readers.\n\nA couple of minor suggestions for enhancem

### Resuming A Previous Task
We can call `run()` or `run_stream()` methods without setting the task again to resume the previous task. The team will continue from where it left off.

In [12]:
# Use the `asyncio.run(Console(reflection_team.run_stream()))` when running in a script.
await Console(reflection_team.run_stream())

---------- primary ----------
Thank you so much for your kind words and positive feedback! I'm glad the poem resonates well in its revised form. If you need any more translations, poetry, or anything else, feel free to ask. I'm here to help!
---------- critic ----------
You're very welcome! I'm glad I could assist you. Your enthusiasm for creating poetry is wonderful, and your willingness to take feedback shows your dedication to your craft. If you have any more requests—be it for translations, poetry, or any other topic—just let me know. I'm here to help you!
---------- primary ----------
Thank you for your encouraging words! I truly appreciate your support and guidance throughout this process. If you have any other requests or if there's anything specific you’d like to explore—be it more poetry, translations, or different topics—please feel free to share. I'm here to assist in any way you need!
---------- critic ----------
You're very welcome! I'm glad to hear that you found the supp

TaskResult(messages=[TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=1910, completion_tokens=47), metadata={}, content="Thank you so much for your kind words and positive feedback! I'm glad the poem resonates well in its revised form. If you need any more translations, poetry, or anything else, feel free to ask. I'm here to help!", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=1974, completion_tokens=63), metadata={}, content="You're very welcome! I'm glad I could assist you. Your enthusiasm for creating poetry is wonderful, and your willingness to take feedback shows your dedication to your craft. If you have any more requests—be it for translations, poetry, or any other topic—just let me know. I'm here to help you!", type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=2030, completion_tokens=63), metadata={}, content="Thank you for your encouraging words! I truly appreciate your s

## Pause for User Input
Often times, team needs additional input from the application (i.e., user) to continue processing the task. We will show two possible ways to do it:

- Set the maximum number of turns such that the team stops after the specified number of turns.

- Use the `HandoffTermination` termination condition.

You can also use custom termination conditions, see Termination Conditions.

### Maximum Number of Turns
This is the simplest way to pause the team for user input. For example, you can set the maximum number of turns to 1 such that the team stops right after the first agent responds. This is useful when you want the user to constantly engage with the team, such as in a chatbot scenario.

Simply set the `max_turns` parameter in the `RoundRobinGroupChat()` constructor.

```python
team = RoundRobinGroupChat([...], max_turns=1)
```

Once the team stops, the turn count will be reset. When you resume the team, it will start from 0 again.

Note that `max_turn` is specific to the team class and is currently only supported by `RoundRobinGroupChat`, `SelectorGroupChat`, and `Swarm`. When used with termination conditions, the team will stop when either condition is met.

### Using Handoff to Pause Team
You can use the `HandoffTermination` termination condition to stop the team when an agent sends a `HandoffMessage` message.

Let’s create a team with a single `AssistantAgent` agent with a handoff setting.

In [13]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import  Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat


# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=aoai_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    system_message="Always transfer to user when you don't know the answer. Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")
termination = handoff_termination | text_termination

# Create a single-agent team.
lazy_agent_team = RoundRobinGroupChat(
    [lazy_agent], termination_condition=termination)

In [14]:
from autogen_agentchat.ui import Console

# Use `asyncio.run(Console(lazy_agent_team.run_stream(task="What is the weather in New York?")))` when running in a script.
await Console(lazy_agent_team.run_stream(task="What is the weather in New York?"))

---------- user ----------
What is the weather in New York?
---------- lazy_assistant ----------
[FunctionCall(id='call_N2E4nDDLB2eO3tVND6HI5XEY', arguments='{}', name='transfer_to_user')]
---------- lazy_assistant ----------
[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_N2E4nDDLB2eO3tVND6HI5XEY', is_error=False)]
---------- lazy_assistant ----------
Transfer to user.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=68, completion_tokens=12), metadata={}, content=[FunctionCall(id='call_N2E4nDDLB2eO3tVND6HI5XEY', arguments='{}', name='transfer_to_user')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='lazy_assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_N2E4nDDLB2eO3tVND6HI5XEY', is_error=False)], type='ToolCallExecutionEvent'), HandoffMessage(source='lazy_assistant', models_usage=None, metadata={}, target='user', content='Transfer to user.', context=[], type='HandoffMessage')], stop_reason='Handoff to user from lazy_assistant detected.')

In [15]:
# Use `asyncio.run(Console(lazy_agent_team.run_stream(task="It is raining in New York.")))` when running in a script.
await Console(lazy_agent_team.run_stream(task="It is raining in New York."))

---------- user ----------
It is raining in New York.
---------- lazy_assistant ----------

---------- lazy_assistant ----------
[FunctionCall(id='call_Q7CZORcmR3VjcnTi7PWdo3RA', arguments='{}', name='transfer_to_user')]
---------- lazy_assistant ----------
[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_Q7CZORcmR3VjcnTi7PWdo3RA', is_error=False)]
---------- lazy_assistant ----------
Transfer to user.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='It is raining in New York.', type='TextMessage'), TextMessage(source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=108, completion_tokens=5), metadata={}, content='', type='TextMessage'), ToolCallRequestEvent(source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=116, completion_tokens=12), metadata={}, content=[FunctionCall(id='call_Q7CZORcmR3VjcnTi7PWdo3RA', arguments='{}', name='transfer_to_user')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='lazy_assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_Q7CZORcmR3VjcnTi7PWdo3RA', is_error=False)], type='ToolCallExecutionEvent'), HandoffMessage(source='lazy_assistant', models_usage=None, metadata={}, target='user', content='Transfer to user.', context=[], type='HandoffMessage')], stop_reason='Handoff to user from lazy_ass